In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.data import Field, BucketIterator,TabularDataset,Iterator
import torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import numpy as np

import random
import math
import time

In [2]:
SEED = 1011

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
tokenize = lambda x: x.split()
SRC = Field(tokenize = tokenize, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

TRG = Field(tokenize = tokenize, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

C:\Users\suman\Miniconda3\envs\py3_env\lib\site-packages\torchtext\data\field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [4]:
unique_fields_train_valid = [('src',SRC),('trg',TRG)]
train = TabularDataset('mt_enghinglish/mt_enghinglish/train.txt',format='tsv',skip_header=False,fields=unique_fields_train_valid)
valid = TabularDataset('mt_enghinglish/mt_enghinglish/dev.txt',format='tsv',skip_header=False,fields=unique_fields_train_valid)
unique_fields_test = [('src',SRC)]
test = TabularDataset('mt_enghinglish/mt_enghinglish/test.txt',format='tsv',skip_header=False,fields=unique_fields_test)

C:\Users\suman\Miniconda3\envs\py3_env\lib\site-packages\torchtext\data\example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
C:\Users\suman\Miniconda3\envs\py3_env\lib\site-packages\torchtext\data\example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [5]:
print(len(train),len(valid),len(test))

8060 942 960


In [6]:
SRC.build_vocab(train,valid)
TRG.build_vocab(train,valid)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
BATCH_SIZE = 32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, valid, test), 
     batch_size = BATCH_SIZE,
     sort_within_batch=True,
     sort_key=lambda x: len(x.src),
     device = device)

C:\Users\suman\Miniconda3\envs\py3_env\lib\site-packages\torchtext\data\iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [9]:
print(len(train_iterator))
print(len(valid_iterator))
print(len(test_iterator))

252
30
30


In [10]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 emb_dim, 
                 hid_dim, 
                 n_layers, 
                 kernel_size, 
                 dropout, 
                 device,
                 max_length = 400):
        super().__init__()
        
        assert kernel_size % 2 == 1, "Kernel size must be odd!"
        
        self.scale = torch.sqrt(torch.FloatTensor([0.5])).to(device)
        
        self.tok_embedding = nn.Embedding(input_dim, emb_dim)
        self.pos_embedding = nn.Embedding(max_length, emb_dim)
        
        self.emb2hid = nn.Linear(emb_dim, hid_dim)
        self.hid2emb = nn.Linear(hid_dim, emb_dim)
        
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = hid_dim, 
                                              out_channels = 2 * hid_dim, 
                                              kernel_size = kernel_size, 
                                              padding = (kernel_size - 1) // 2)
                                    for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [batch size, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        #create position tensor
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(device)
        
        #pos = [0, 1, 2, 3, ..., src len - 1]
        
        #pos = [batch size, src len]
        
        #embed tokens and positions
        tok_embedded = self.tok_embedding(src)
        pos_embedded = self.pos_embedding(pos)
        
        #tok_embedded = pos_embedded = [batch size, src len, emb dim]
        
        #combine embeddings by elementwise summing
        embedded = self.dropout(tok_embedded + pos_embedded)
        
        #embedded = [batch size, src len, emb dim]
        
        #pass embedded through linear layer to convert from emb dim to hid dim
        conv_input = self.emb2hid(embedded)
        
        #conv_input = [batch size, src len, hid dim]
        
        #permute for convolutional layer
        conv_input = conv_input.permute(0, 2, 1) 
        
        #conv_input = [batch size, hid dim, src len]
        
        #begin convolutional blocks...
        
        for i, conv in enumerate(self.convs):
        
            #pass through convolutional layer
            conved = conv(self.dropout(conv_input))

            #conved = [batch size, 2 * hid dim, src len]

            #pass through GLU activation function
            conved = F.glu(conved, dim = 1)

            #conved = [batch size, hid dim, src len]
            
            #apply residual connection
            conved = (conved + conv_input) * self.scale

            #conved = [batch size, hid dim, src len]
            
            #set conv_input to conved for next loop iteration
            conv_input = conved
        
        #...end convolutional blocks
        
        #permute and convert back to emb dim
        conved = self.hid2emb(conved.permute(0, 2, 1))
        
        #conved = [batch size, src len, emb dim]
        
        #elementwise sum output (conved) and input (embedded) to be used for attention
        combined = (conved + embedded) * self.scale
        
        #combined = [batch size, src len, emb dim]
        
        #print("encoder_finished")
        
        return conved, combined

In [11]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 emb_dim, 
                 hid_dim, 
                 n_layers, 
                 kernel_size, 
                 dropout, 
                 trg_pad_idx, 
                 device,
                 max_length = 400):
        super().__init__()
        
        self.kernel_size = kernel_size
        self.trg_pad_idx = trg_pad_idx
        
        self.scale = torch.sqrt(torch.FloatTensor([0.5])).to(device)
        
        self.tok_embedding = nn.Embedding(output_dim, emb_dim)
        self.pos_embedding = nn.Embedding(max_length, emb_dim)
        
        self.emb2hid = nn.Linear(emb_dim, hid_dim)
        self.hid2emb = nn.Linear(hid_dim, emb_dim)
        
        self.attn_hid2emb = nn.Linear(hid_dim, emb_dim)
        self.attn_emb2hid = nn.Linear(emb_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim, output_dim)
        
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = hid_dim, 
                                              out_channels = 2 * hid_dim, 
                                              kernel_size = kernel_size)
                                    for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
      
    def calculate_attention(self, embedded, conved, encoder_conved, encoder_combined):
        
        #embedded = [batch size, trg len, emb dim]
        #conved = [batch size, hid dim, trg len]
        #encoder_conved = encoder_combined = [batch size, src len, emb dim]
        
        #permute and convert back to emb dim
        conved_emb = self.attn_hid2emb(conved.permute(0, 2, 1))
        
        #conved_emb = [batch size, trg len, emb dim]
        
        combined = (conved_emb + embedded) * self.scale
        
        #combined = [batch size, trg len, emb dim]
                
        energy = torch.matmul(combined, encoder_conved.permute(0, 2, 1))
        
        #energy = [batch size, trg len, src len]
        
        attention = F.softmax(energy, dim=2)
        
        #attention = [batch size, trg len, src len]
            
        attended_encoding = torch.matmul(attention, encoder_combined)
        
        #attended_encoding = [batch size, trg len, emd dim]
        
        #convert from emb dim -> hid dim
        attended_encoding = self.attn_emb2hid(attended_encoding)
        
        #attended_encoding = [batch size, trg len, hid dim]
        
        #apply residual connection
        attended_combined = (conved + attended_encoding.permute(0, 2, 1)) * self.scale
        
        #attended_combined = [batch size, hid dim, trg len]
        
        return attention, attended_combined
        
    def forward(self, trg, encoder_conved, encoder_combined):
        
        #trg = [batch size, trg len]
        #encoder_conved = encoder_combined = [batch size, src len, emb dim]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
            
        #create position tensor
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(device)
        
        #pos = [batch size, trg len]
        
        #embed tokens and positions
        tok_embedded = self.tok_embedding(trg)
        pos_embedded = self.pos_embedding(pos)
        
        #tok_embedded = [batch size, trg len, emb dim]
        #pos_embedded = [batch size, trg len, emb dim]
        
        #combine embeddings by elementwise summing
        embedded = self.dropout(tok_embedded + pos_embedded)
        
        #embedded = [batch size, trg len, emb dim]
        
        #pass embedded through linear layer to go through emb dim -> hid dim
        conv_input = self.emb2hid(embedded)
        
        #conv_input = [batch size, trg len, hid dim]
        
        #permute for convolutional layer
        conv_input = conv_input.permute(0, 2, 1) 
        
        #conv_input = [batch size, hid dim, trg len]
        
        batch_size = conv_input.shape[0]
        hid_dim = conv_input.shape[1]
        
        for i, conv in enumerate(self.convs):
        
            #apply dropout
            conv_input = self.dropout(conv_input)
        
            #need to pad so decoder can't "cheat"
            padding = torch.zeros(batch_size, 
                                  hid_dim, 
                                  self.kernel_size - 1).fill_(self.trg_pad_idx).to(device)
                
            padded_conv_input = torch.cat((padding, conv_input), dim = 2)
        
            #padded_conv_input = [batch size, hid dim, trg len + kernel size - 1]
        
            #pass through convolutional layer
            conved = conv(padded_conv_input)

            #conved = [batch size, 2 * hid dim, trg len]
            
            #pass through GLU activation function
            conved = F.glu(conved, dim = 1)

            #conved = [batch size, hid dim, trg len]
            
            #calculate attention
            attention, conved = self.calculate_attention(embedded, 
                                                         conved, 
                                                         encoder_conved, 
                                                         encoder_combined)
            
            #attention = [batch size, trg len, src len]
            
            #apply residual connection
            conved = (conved + conv_input) * self.scale
            
            #conved = [batch size, hid dim, trg len]
            
            #set conv_input to conved for next loop iteration
            conv_input = conved
            
        conved = self.hid2emb(conved.permute(0, 2, 1))
         
        #conved = [batch size, trg len, emb dim]
            
        output = self.fc_out(self.dropout(conved))
        
        #output = [batch size, trg len, output dim]
        
        #print("decoder_finished")
            
        return output, attention

In [12]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len - 1] (<eos> token sliced off the end)
           
        #calculate z^u (encoder_conved) and (z^u + e) (encoder_combined)
        #encoder_conved is output from final encoder conv. block
        #encoder_combined is encoder_conved plus (elementwise) src embedding plus 
        #  positional embeddings 
        encoder_conved, encoder_combined = self.encoder(src)
            
        #encoder_conved = [batch size, src len, emb dim]
        #encoder_combined = [batch size, src len, emb dim]
        
        #calculate predictions of next words
        #output is a batch of predictions for each word in the trg sentence
        #attention a batch of attention scores across the src sentence for 
        #  each word in the trg sentence
        output, attention = self.decoder(trg, encoder_conved, encoder_combined)
        
        #output = [batch size, trg len - 1, output dim]
        #attention = [batch size, trg len - 1, src len]
        
        return output, attention

In [13]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
EMB_DIM = 256
HID_DIM = 512 # each conv. layer has 2 * hid_dim filters
ENC_LAYERS = 10 # number of conv. blocks in encoder
DEC_LAYERS = 10 # number of conv. blocks in decoder
ENC_KERNEL_SIZE = 3 # must be odd!
DEC_KERNEL_SIZE = 3 # can be even or odd
ENC_DROPOUT = 0.25
DEC_DROPOUT = 0.25
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
    
enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM, ENC_LAYERS, ENC_KERNEL_SIZE, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, DEC_LAYERS, DEC_KERNEL_SIZE, DEC_DROPOUT, TRG_PAD_IDX, device)

enc = enc.to(device)
dec = dec.to(device)

model = Seq2Seq(enc, dec).to(device)

In [14]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)
model.apply(initialize_weights);

In [15]:
optimizer = torch.optim.Adam(model.parameters())

In [16]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [17]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        #print(src.shape,trg.shape)
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
        
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
        
        output_dim = output.shape[-1]
        
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
        
        
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [18]:
#train_loss = train(model, train_iterator, optimizer, criterion, 1)

In [19]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output, _ = model(src, trg[:,:-1])
        
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]

            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)

            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [20]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [23]:
N_EPOCHS = 10
CLIP = 0.1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'cm-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 14s
	Train Loss: 3.399 | Train PPL:  29.931
	 Val. Loss: 5.599 |  Val. PPL: 270.057
Epoch: 02 | Time: 0m 14s
	Train Loss: 3.125 | Train PPL:  22.767
	 Val. Loss: 5.589 |  Val. PPL: 267.354
Epoch: 03 | Time: 0m 14s
	Train Loss: 2.879 | Train PPL:  17.798
	 Val. Loss: 5.704 |  Val. PPL: 299.948
Epoch: 04 | Time: 0m 14s
	Train Loss: 2.664 | Train PPL:  14.360
	 Val. Loss: 5.721 |  Val. PPL: 305.106
Epoch: 05 | Time: 0m 14s
	Train Loss: 2.489 | Train PPL:  12.053
	 Val. Loss: 5.780 |  Val. PPL: 323.829
Epoch: 06 | Time: 0m 14s
	Train Loss: 2.338 | Train PPL:  10.362
	 Val. Loss: 5.844 |  Val. PPL: 345.156
Epoch: 07 | Time: 0m 14s
	Train Loss: 2.201 | Train PPL:   9.033
	 Val. Loss: 6.057 |  Val. PPL: 427.301
Epoch: 08 | Time: 0m 14s
	Train Loss: 2.083 | Train PPL:   8.026
	 Val. Loss: 5.978 |  Val. PPL: 394.622
Epoch: 09 | Time: 0m 14s
	Train Loss: 1.979 | Train PPL:   7.232
	 Val. Loss: 6.095 |  Val. PPL: 443.433
Epoch: 10 | Time: 0m 14s
	Train Loss: 1.899 | Train PPL

In [22]:
model.load_state_dict(torch.load('cm-model.pt'))

test_loss = evaluate(model, valid_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.489 | Test PPL: 241.951 |


In [24]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 400):

    model.eval()

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    with torch.no_grad():
        encoder_conved, encoder_combined = model.encoder(src_tensor)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, encoder_conved, encoder_combined)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [44]:
import pandas as pd
fin = open("mt_enghinglish/mt_enghinglish/test.txt",encoding='utf-8')
fout = open("mt_enghinglish/mt_enghinglish/predictions.txt","w+",encoding='utf-8')
count=0
for line in fin:
    count+=1
    print(count)
    line = line.replace("\n","")
    line = line.split()
    trg_sentence,attention = translate_sentence(line,SRC,TRG,model,device=device)
    print(" ".join(trg_sentence[:-1]))
    fout.write(" ".join(trg_sentence[:-1])+"\n")

1
haan, ye tho is scenes ka tarah hein, do do dusron ko lethe
2
han ye good hai
3
mujhe matlab hai m hang legend
4
hello
5
mujhe bhi. aur jyada movie twist kiya
6
mujhe lagta hai unhone uski phir defense defense defense defense cheating ke liye disqualify ho jayega
7
isake movie tho koi time hi hota ho kaise sa kid kaise the kevin aur kevin ko apni scene le gaya aur kevin aur kevin ko apni scene le gaya aur kevin aur kevin ko apni scene add kar rahe the aur kevin ko apni scene aur kevin ko
8
but uske paas dekhne ko dekhne ko dekhne ko padegi
9
haan, ab magar bahut achhi hi hi ye achi nahi. mere sath mere sath
10
nice!
11
mujhe bhi ye bhi pasand hein book ka next acchi comic
12
haan, koi yeh original aur sequels hi sequels hi
13
oh, aur maleficent ek dark bat hai.
14
kya jeenifer vitamin overdose mein, uska uska god nahi hein mujhe ye part yaad hai
15
ye ek acchi scene hein jab wo tho yeh streaming right hai jab wo wo ek general general jaisa aur aur wo john aur ya kuch animated ho rahe

hi! kya tumhe pasand hein ki kaise reeves ne bahut kamal kiya hein
142
kya aap is movie ka naam baata sakte ho aap koi koi koi film movie dekhi hai?
143
mujhe lagta hai usne bahut achcha kaam kiya
144
perfect! hai
145
pixar
146
mujhe tho sure ke sath se kam tha usne unhone ache nahi diya
147
nahi par but mujhe lagta hai ki ye is actor ko ek acche acche nahin hain ve kaise acha hai tumhe tumhe accha ho
148
sure!
149
user 2 chat cheet chodd raha hai
150
oh wow jan accha ha aap bahut achha
151
aap ka naam soch raha ho cheating ka cheating universe ke liye
152
sometimes mujhe lagta hai ki mere family ek ek behatar strike saktha hoga
153
mujhe laga ye super romantic romantic romantic dekhi tha
154
mujhe sehmat hoon is avengers mein bahut ache movie hein
155
mera patni ne ek bada improovament dikhti hai christopher nolan mein ek ek naya philm se ek maja movie hai agar wo ye naya movie se use maja sakta hai ki wo ek naya movies se dekha hai
156
ek thora similar hai ki ye song us song ko direc

i agree, ye vakahi ki ek ek thriller film pasand hai. jo maanate hue land par sangeet par portray kiya hai.
284
han yar, wo awesome hein
285
haan! tho ek ek achchhee tarah hai, yah ek comedy movie hai ye ye best movie hai ye wo best movie hai.
286
mujhe sehmat hoon is avengers mein bahut ache movie hein
287
mujhe frozen ki movie pasand hai, yeh mujhe mere bachche bhi pasand hai aur main mere bachche bhi
288
ye ye 2002 mein hai
289
haay, kaise raha ho?
290
aur sabhi action hein
291
mujhe bet lga ki a din din
292
hello!
293
mujhe assume karta hai ki nolan ne direct kiya tha
294
hello!
295
lol mujhe matlab play play kiya tha to ye ek time bhi yaad tha
296
thanks ke bare mein batane ke liye thanks
297
sahi
298
kaise se
299
ye ye 2016 mein bani hai khud tak daraane mein
300
mujhe lagta hai unhone uski phir defense defense defense defense cheating ke liye disqualify ho jayega
301
kya wo kuch back hone mei release tha?
302
mujhe home alone pasand aaya jab ye isey mazedaar hua?
303
i surprise 

mujhe bhi bahut accha kaam kaam dena dena accha kaam karne ke liye kuch kuch experience se run kar raha tha shayad mujhe ek samay samay se try kar raha tha
413
ye kaafi pretty cute thi ki main aamataur ki main time story ki based hai jo main ek time book ki based hai jo ek ek kisi ka tarah ki book ko ek ek hero kee tarah kee tarah ki ek ek hero kee tarah kee taraf ki tarah
414
hello!
415
maine ye movie ki suna hai lekin nahi remember remember hai
416
mujhe movie acchi aachi hai, american superhero film film based hai movies meri marvel marvel comics aur villains hai
417
ye tho funny ke baare mein hai
418
kya topic bhi
419
maleficent basically ek species stereotype ko determine karne ke liye flak lega
420
pretty good score hai
421
ye good he
422
mujhe ye clean up laga jo kaise hook me lol
423
ohh sorry
424
mujhe tho utna ek scene scene pasand hai jo seamless scene the
425
user 2 chat cheet chodd raha hai
426
tho hajar tarah ke liye kuch alag hota hain ab bahut different different hain
4

mujhe hamesha, bhul jaathe tha as dar ki bare ki ek ek philm movie se ek baare mein ek ek philm kee baare mein ek suna hai.
532
mein ke bat mein
533
key scenes ke bare mein kya scene kya sochte ho?
534
aap movie ko jim carrey movie enjoy karta hai?
535
jarror
536
kya tumhe lagta hai ki ek acchi philm hai.tereko hain?
537
mujhe lagta hai ye ek badhiya philm thee, movie kya aapako lagta hai
538
hello!
539
haan yes
540
mujhe ise ise dekhane ko dekhana ise dekhana jaante hai mujhe lagata hai ki yah ek mother ko daayana ko koshish karta hai
541
nick's dimaag ke tarah karata hai
542
slinky dog bhi ek mera favorite feel tha, mujhe mein isako bhi tab, tho :)
543
well, ye ek ek mahan film hai.
544
mujhe ye bani se ise kis gane se se ek set jagah tha
545
ye bahut funny lagta hai. mujhe morgan freeman bahut accha lagta hai.
546
key scenes ke bare mein kya scene kya sochte ho?
547
haan, ye true story ke tarah story hai but yeh kuch ek fighting movie hai but kuch ek ache acting hai par bahut ek acc

haan, ye interesting hai jab aur us log ko us widow? prashikshit dekhayi gaya
676
avengers tho avengers kaise the
677
well wo apni history se second karane ke liye doosaree yaatra mein jodane mein ek maanyata kar rahe the aur wo ek ignor karne ki wo ek ignor karne ki
678
main bhi terrance hi kuch lagtha,.. aur tho tho koi koi hi bhi
679
the ka mom ka end par lekar bachana ho gaya
680
mujhe yaad hai kaafi ek baat karne se
681
mujhe lagta hai ki mera personal comedy movies movie ko mera favorite films ko bhi mera favorite film hein ki mein is movie ko movies ko yaad hein aur mein is movie ko bhi ko mein film ko bhi mein tho mein tho yeh film ko film ko yaad
682
benedict benedict characters mein jay actor hein pramukh role kaun nibhaata hein
683
haan tho uska aur koi badi jagah amanda hota hai isme mean girls ke baad
684
but maleficent sleeping beauty movie hai
685
hello
686
mujhe bhi part pasand hein
687
kya
688
kaise kaise pyaar se pyaar karata hain phir?
689
mujhe unamen pasand hein, u

kya main aur keira knightly se pasand nahi karta.
807
ye ke baare mein hai jo use ek scorcher hai.
808
kya maine nahi dekhi
809
mujhe lagta hai sab 3 achhe hote hein
810
yes. voh tho ek aise movie nahin hoga jo voh ek story.
811
mujhe us dream mein kaise lagta hai ki david stern mind mein character ki jo real dream mein hai.
812
wait ke case main mujhe patha hein jab sorry
813
bahut
814
american tho classic ek comedy hein, aur yeh achi comedy hein aur yeh sab sab ke liye tho yeh sab aise hi hein
815
yeh tho sach hein
816
i agree, ye vakahi ki ek ek thriller film pasand hai. jo maanate hue land par sangeet par portray kiya hai.
817
haay, kaise raha ho?
818
funny ka message hein, movie tho is tarah ko pasand nahi saktha aur mujhe ek hero ko follow karo aur main log kehne karo ya koi conversation kehne karo ya koi ko jaante ho ya log log kehne karte hai ya ek baar chat karte hai ya koi log
819
ye tho 2017 ki achhi drama movie hai
820
mujhe to information ke liye try kar raha tha
821
nahi


mein kuch kuch pata hai ki wo sab patha nahi chahta kyunki wo ek apane dream ko ek familiar nahi hota aur wo ek right jaana le rahe hai
944
par uski favorite samay
945
mujhe fantasy pasand hein
946
haan, aap tho ee tho missed hi hey
947
han mujhe laga ki uska ek details aur mention kiya aur us log ek pasand kaam kar sakti tha
948
ache boxers feel feel hein
949
main character kon hai?
950
main sahamat hoon
951
ye interesting sound karta hai? kya tumne movie kabhi dekha hai?
952
tumhe movie ke baare mein kya sochte hein? movie movie mein is movie mein ache part hein
953
tumhe kya lagta hai ki the best scene konsa scene
954
ye ke sath par
955
as se aalochak critic film hai, haalaanki yeh ek major philm film hai, although yah ek general ank singer hai.
956
agar mujhe uski bhi pasand hein, yeh tho yeh bhi bhi bhi bhi bhi ab bhi bhi bhi ab bhi bhi bhi bhi bhi bhi bhi ab bhi bhi bhi bhi bhi bhi
957
ye tho bura baat nahi hein kya tum kabhi kabhi dekhne ke liye convince kar rahe ho
958
han
959


In [45]:
fout.close()